In [6]:
import kfp
from kfp import dsl

# Step 1: Download dataset from Roboflow
@dsl.component(packages_to_install=['opencv-python-headless', 'ultralytics', 'roboflow'], base_image="python:3.9")
def download_dataset(api_key: str, workspace: str, project_name: str, version_number: int) -> str:
    import os
    import subprocess
    from roboflow import Roboflow
    
    rf = Roboflow(api_key=api_key)
    project = rf.workspace(workspace).project(project_name)
    version = project.version(version_number)
    dataset_path = version.download("yolov11")
    
    return dataset_path

# Step 2: Train YOLO Model
@dsl.component(packages_to_install=['opencv-python-headless', 'ultralytics', 'roboflow'], base_image="python:3.9")
def train_model(dataset_path: str) -> str:
    import os
    import subprocess

    from ultralytics import YOLO
    
    model = YOLO("yolo11n.pt")
    results = model.train(
        data=os.path.join(dataset_path, "data.yaml"),
        epochs=100,
        imgsz=640,
        batch=16,
        device="cuda"
    )
    
    best_model_path = os.path.join("runs/train", results.save_dir, "weights/best.pt")
    return best_model_path

# Define the Kubeflow pipeline
@dsl.pipeline(name="roboflow-yolo-training")
def roboflow_pipeline(api_key: str, workspace: str, project_name: str, version_number: int):
    dataset_task = download_dataset(api_key=api_key, workspace=workspace, project_name=project_name, version_number=version_number)
    train_task = train_model(dataset_path=dataset_task.output)

# Compile the pipeline
if __name__ == "__main__":
    kfp.compiler.Compiler().compile(roboflow_pipeline, "roboflow_pipeline.yaml")
